In [32]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 1000000)
pd.set_option('display.max_rows', 1000000)

In [33]:
#Remove Na values
data = pd.read_csv('Amazon.csv')
print(data.shape)
data.Summary = data.Summary.fillna('0')
data.ProfileName = data.ProfileName.fillna(' ')

(455000, 13)


In [34]:
#Various text features from review text
data['hasSC'] = data['Text'].str.contains(';')
data['hasbroken'] = data['Text'].str.contains('broken')
data['hasdidnt'] = data['Text'].str.contains('didn\'t')
data['hasperfect'] = data['Text'].str.contains('perfect')
data['haslove'] = data['Text'].str.contains('love')
data['hasEP'] = data['Text'].str.contains('!')
data['hasdash'] = data['Text'].str.contains('-')
data['hasChocolate'] = data['Text'].str.contains('chocolate')
data['hasCoffee'] = data['Text'].str.contains('coffee')
data['hasRec'] = data['Text'].str.contains('recommend')
data['hasTerrible'] = data['Text'].str.contains('terrible')
data['sumEP'] = data['Summary'].str.contains('!')
data['sumPeriod'] = data['Summary'].str.contains('.')
data['nameSpace'] = data['ProfileName'].str.contains(' ')
data['sumLen'] = data['Summary'].str.len()
data['reviewLen'] = data['Text'].str.len()
#Has the review writer written more than one review
data['duplicateIds'] = data.UserId.duplicated()
data['nameLen'] = data['ProfileName'].str.len()
#the difference in character length between summary and text
data['sumtexdif'] = data['Text'].str.len() - data['Summary'].str.len()
data.sumtexdif[data.sumtexdif < 0] = 0

data['sumComma'] = data['Summary'].str.contains(',')
data['sumColon'] = data['Summary'].str.contains(':')
data['sumSC'] = data['Summary'].str.contains(';')
data['sumDash'] = data['Summary'].str.contains('-')
data['hasComma'] = data['Text'].str.contains(',')
data['hasColon'] = data['Text'].str.contains(':')
#Include values of reviews written per user and per product
data['ProductCounts'] = data.groupby('ProductId')['ProductId'].transform('count')
data['UserCounts'] = data.groupby('UserId')['UserId'].transform('count')

/Users/ianssmith/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
###Reshaping and combining features for model input
XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)

hasSC = data.iloc[:, 13].values.reshape(data.shape[0], 1)
hasbroken = data.iloc[:, 14].values.reshape(data.shape[0], 1)
hasdidnt = data.iloc[:, 15].values.reshape(data.shape[0], 1)
hasperfect = data.iloc[:, 16].values.reshape(data.shape[0], 1)
haslove = data.iloc[:, 17].values.reshape(data.shape[0], 1)
hasEP = data.iloc[:, 18].values.reshape(data.shape[0], 1)
hasdash = data.iloc[:, 19].values.reshape(data.shape[0], 1)
hasChocolate = data.iloc[:, 20].values.reshape(data.shape[0],1)
hasCoffee = data.iloc[:, 21].values.reshape(data.shape[0],1)
hasRec = data.iloc[:, 22].values.reshape(data.shape[0],1)
hasTerrible = data.iloc[:, 23].values.reshape(data.shape[0],1)
sumEP = data.iloc[:, 24].values.reshape(data.shape[0],1)
sumPeriod = data.iloc[:, 25].values.reshape(data.shape[0],1)
nameSpace = data.iloc[:, 26].values.reshape(data.shape[0],1)
sumLen = data.iloc[:, 27].values.reshape(data.shape[0],1)
XreviewLen = data.iloc[:, 28].values.reshape(data.shape[0], 1)
XIddups = data.iloc[:, 29].values.reshape(data.shape[0], 1)
nameLen = data.iloc[:, 30].values.reshape(data.shape[0], 1)
sumtexdif = data.iloc[:, 31].values.reshape(data.shape[0], 1)
sumComma = data.iloc[:, 32].values.reshape(data.shape[0],1)
sumColon = data.iloc[:, 33].values.reshape(data.shape[0],1)
sumSC = data.iloc[:, 34].values.reshape(data.shape[0],1)
sumDash = data.iloc[:, 35].values.reshape(data.shape[0],1)
hasComma = data.iloc[:, 36].values.reshape(data.shape[0],1)
hasColon = data.iloc[:, 37].values.reshape(data.shape[0],1)
ProductCounts = data.iloc[:, 38].values.reshape(data.shape[0],1)
UserCounts = data.iloc[:, 39].values.reshape(data.shape[0],1)

Xtoadd = np.concatenate(( hasSC, hasbroken, hasdidnt, hasperfect, 
                         haslove, hasEP, hasdash, XScore, XreviewLen,
                         XIddups, hasChocolate, hasCoffee, hasRec,
                         hasTerrible, sumtexdif, sumEP, sumPeriod,
                         sumLen, nameLen, nameSpace, sumComma, sumColon,
                         sumSC, sumDash, hasComma, hasColon , ProductCounts, UserCounts), axis=1)



In [36]:
#Function to show model results
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training tet:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [37]:
import nltk.stem
from sklearn.feature_extraction.text import CountVectorizer
english_stemmer = nltk.stem.SnowballStemmer('english')

In [38]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [39]:
vectorizer = StemmedCountVectorizer(min_df=1,
stop_words='english')

In [40]:
#Run stemming vectorizer on text based elements of data to add into feature set
Xstemmer = vectorizer.fit_transform(data.Text)
XstemmerSum = vectorizer.fit_transform(data.Summary)
Xstemname = vectorizer.fit_transform(data.ProfileName)
Xstemuid = vectorizer.fit_transform(data.UserId)
Xstempid = vectorizer.fit_transform(data.ProductId)

In [41]:
#Convert features
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)

In [42]:
#Combine features
Xfinal = hstack([Xstemmer, XstemmerSum, Xstemname, Xstemuid, Xstempid, XtoaddSparse])
X = csr_matrix(Xfinal)

In [43]:
#Specify y value
y = data.iloc[:, 12].values

In [44]:
#Split data for training and testing
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [45]:
#Train and predict; Print results
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.fit(X_train, y_train).predict(X_train)
y_pred_test = clf.predict(X_test)
print_results()

Error rate on training set: 
0.392913657771
Accuracy rate on training set: 
0.607086342229
True positive rate on training tet:
0.647795763733
**************
Error rate on test set: 
0.408131868132
Accuracy rate on test set: 
0.591868131868
True positive rate on test set
0.630758469072
True negative rate on test set
0.588791474627
